In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import ML libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import pickle # to save and load pre-trained models

### Read in Data

In [3]:
trades_df = pd.read_csv('buy_signals_backtest_ML_July-Aug2020_volume.dat')
trades_df.columns

Index(['time_curr', 'symbol', 'price', 'pattern', 'origin', 'ranging',
       'd_ranging', 'lower', 'upper', 'middle', 'd_lower', 'd_upper',
       'd_middle', 'ema_10', 'ema_200', 'd_ema_10', 'd_ema_200', 'k_15',
       'd_15', 'd_k_15', 'd_d_15', 'stoch_cond_5', 'candle_color', 'volume',
       'ntrades', 'tb_volume', 'vol_inc', 'dist_to_BB', 'profit', 'elapsed',
       'min_price', 'max_price', 'profit_s15', 'elapsed_s15', 'min_price_s15',
       'max_price_s15', 'Unnamed: 36'],
      dtype='object')

#### Create target variable

In [4]:
profitability = [0 if item < 0 else 1 for item in trades_df['profit']]
trades_df['profitability'] = profitability
profitability_s15 = [0 if item < 0 else 1 for item in trades_df['profit_s15']]
trades_df['profitability_s15'] = profitability_s15

Convert 'time_curr' to `datetime` object

In [5]:
trades_df['time_curr'] = pd.to_datetime(trades_df['time_curr'])

# Sort the dataframe by the date:
trades_df.sort_values(by=['time_curr'], inplace=True)
trades_df.columns

Index(['time_curr', 'symbol', 'price', 'pattern', 'origin', 'ranging',
       'd_ranging', 'lower', 'upper', 'middle', 'd_lower', 'd_upper',
       'd_middle', 'ema_10', 'ema_200', 'd_ema_10', 'd_ema_200', 'k_15',
       'd_15', 'd_k_15', 'd_d_15', 'stoch_cond_5', 'candle_color', 'volume',
       'ntrades', 'tb_volume', 'vol_inc', 'dist_to_BB', 'profit', 'elapsed',
       'min_price', 'max_price', 'profit_s15', 'elapsed_s15', 'min_price_s15',
       'max_price_s15', 'Unnamed: 36', 'profitability', 'profitability_s15'],
      dtype='object')

#### Check what was the profitability during backtest

In [6]:
time_span = trades_df.time_curr.iloc[-1] - trades_df.time_curr.iloc[0]
print(f"Start trade: {trades_df.time_curr.iloc[0]}, end trade: {trades_df.time_curr.iloc[-1]} ")
print(f"Total profitability in {time_span}: ", trades_df['profitability'].sum()/len(trades_df['profitability']) )
print("total n trades: ", len(trades_df['profitability']))
print(f"Total profitability in {time_span} with 0.15 stop loss: ", trades_df['profitability_s15'].sum()/len(trades_df['profitability_s15']) )
print("total n trades: ", len(trades_df['profitability_s15']))

Start trade: 2020-07-02 00:00:00, end trade: 2020-10-25 11:03:00 
Total profitability in 115 days 11:03:00:  0.6163383874167493
total n trades:  14114
Total profitability in 115 days 11:03:00 with 0.15 stop loss:  0.4427518775683718
total n trades:  14114


### Clean data

In [7]:
# Remove useless column
trades_df.drop(['Unnamed: 36'], axis=1, inplace=True)
# Fill Nans
trades_df.fillna(-9999, inplace=True)


In [8]:
# Remove outliers
trades_df = trades_df[trades_df.d_ema_200 != -9999]

### Now we start preparing the dataset to train a model using various Machine Learning methods

In [9]:
# Features:
X = trades_df.drop(['time_curr','symbol','profit',
                    'stoch_cond_5','vol_inc',
       'elapsed', 'min_price', 'max_price', 'profit_s15', 'elapsed_s15',
       'min_price_s15', 'max_price_s15',  'profitability',
       'profitability_s15'], axis=1)
X.columns

Index(['price', 'pattern', 'origin', 'ranging', 'd_ranging', 'lower', 'upper',
       'middle', 'd_lower', 'd_upper', 'd_middle', 'ema_10', 'ema_200',
       'd_ema_10', 'd_ema_200', 'k_15', 'd_15', 'd_k_15', 'd_d_15',
       'candle_color', 'volume', 'ntrades', 'tb_volume', 'dist_to_BB'],
      dtype='object')

#### Scale small values by a factor...

In [10]:
for col in ['price','lower','upper','middle','d_lower', 'd_upper', 'd_middle',
            'ema_10', 'ema_200','d_ema_10', 'd_ema_200' ]: X[col] = X[col]*1e5

#### Categorical encoding

In [11]:
# For now go with the simplest One-Hot encoding provided by pandas:
X = pd.get_dummies(X)
X.head()

,price,ranging,d_ranging,lower,upper,middle,d_lower,d_upper,d_middle,ema_10,...,dist_to_BB,pattern_Bullish eng.,pattern_Doji,pattern_Hammer,pattern_Harami,pattern_no,origin_lower,origin_upper,candle_color_green,candle_color_red
11666,2.241,5.444,0.011,2.226,2.343,2.284,-0.001,-0.0,-0.0,2.263,...,1.932,0,0,0,0,1,1,0,0,1
11668,2.237,5.599,0.010,2.220,2.340,2.280,-0.000,-0.0,-0.0,2.258,...,1.924,0,0,0,0,1,1,0,1,0
10180,4.023,4.460,-0.014,3.926,4.110,4.018,0.001,-0.0,0.0,4.025,...,2.150,0,0,0,0,1,0,1,0,1
405,4.023,4.460,-0.014,3.926,4.110,4.018,0.001,-0.0,0.0,4.025,...,2.150,0,0,0,0,1,0,1,0,1
11670,2.209,5.350,0.011,2.191,2.304,2.248,-0.000,-0.0,-0.0,2.222,...,1.752,0,1,0,0,0,1,0,1,0


In [12]:
# target variable
y = trades_df.profitability_s15

### Split into train, validation, and test set

![Split Data](img/split_data.png)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=99)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=99)

In [14]:
# Make sure that the data were split correctly
for dataset in [y_train, y_val, y_test]:
    print(round(len(dataset) / len(y), 2))

0.6
0.2
0.2


In [15]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

### Write a function to compare models performance

In [16]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

# Algorithms

# 1. Logistic regression

Discover hyperparamaters

In [17]:
LogisticRegression()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

### Read in data

![CV](img/CV.png)
![Cross-Val](img/Cross-Val.png)

### Hyperparameter tuning

![C](img/c.png)

In [18]:
# Find the best model by tuning C using GridSearch CV:

lr = LogisticRegression()
parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

cv = GridSearchCV(lr, parameters, cv=5, verbose=5, n_jobs=-1)
cv.fit(X_train, y_train.values.ravel())

print_results(cv)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed:    6.4s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    6.9s finished


BEST PARAMS: {'C': 1000}

0.56 (+/-0.006) for {'C': 0.001}
0.567 (+/-0.005) for {'C': 0.01}
0.578 (+/-0.005) for {'C': 0.1}
0.602 (+/-0.018) for {'C': 1}
0.617 (+/-0.02) for {'C': 10}
0.618 (+/-0.02) for {'C': 100}
0.618 (+/-0.023) for {'C': 1000}


In [35]:
# Save the best model
#log_reg_best = cv.best_estimator_

with open('LR_best.pickle', 'wb') as f:
    pickle.dump(log_reg_best, f)

log_reg_best

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

# 2. Support Vector Machines (SVM)

In [20]:
from sklearn.svm import SVC

In [21]:
SVC()

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

### Hyperparameter tuning

![c](img/c_svm.png)

In [22]:
svc = SVC()
parameters = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10]
}

cv = GridSearchCV(svc, parameters, cv=5, verbose=5, n_jobs=-1)
cv.fit(X_train, y_train.values.ravel())

print_results(cv)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done  22 out of  30 | elapsed: 31.6min remaining: 11.5min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 68.9min finished


BEST PARAMS: {'C': 10, 'kernel': 'linear'}

0.562 (+/-0.005) for {'C': 0.1, 'kernel': 'linear'}
0.56 (+/-0.0) for {'C': 0.1, 'kernel': 'rbf'}
0.593 (+/-0.017) for {'C': 1, 'kernel': 'linear'}
0.571 (+/-0.019) for {'C': 1, 'kernel': 'rbf'}
0.603 (+/-0.018) for {'C': 10, 'kernel': 'linear'}
0.56 (+/-0.013) for {'C': 10, 'kernel': 'rbf'}


In [34]:
# Save the best model
#svm_best = cv.best_estimator_

with open('SVM_best.pickle', 'wb') as f:
    pickle.dump(svm_best, f)

svm_best

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

# 3. Multilayer Perceptron

Discover hyperparameters

In [24]:
from sklearn.neural_network import MLPRegressor, MLPClassifier

print(MLPRegressor())
print(MLPClassifier())

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)


### Hyperparameter tuning

![hidden layer](img/hidden_layers.png)

In [26]:
mlp = MLPClassifier()
parameters = {
    'hidden_layer_sizes': [(10,), (50,), (100,)],
    'activation': ['relu', 'tanh', 'logistic'],
    'learning_rate': ['constant', 'invscaling', 'adaptive']
}

cv = GridSearchCV(mlp, parameters, cv=5, n_jobs=-1, verbose=10)
cv.fit(X_train, y_train.values.ravel())

print_results(cv)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:   39.1s finished


BEST PARAMS: {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'invscaling'}

0.565 (+/-0.022) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'constant'}
0.573 (+/-0.036) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'invscaling'}
0.555 (+/-0.044) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'adaptive'}
0.547 (+/-0.051) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'constant'}
0.549 (+/-0.055) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'invscaling'}
0.547 (+/-0.056) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'adaptive'}
0.54 (+/-0.088) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}
0.555 (+/-0.051) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'invscaling'}
0.556 (+/-0.023) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learn

In [36]:
#mlp_best = cv.best_estimator_

with open('MLP_best.pickle', 'wb') as f:
    pickle.dump(mlp_best, f)

mlp_best

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10,), learning_rate='invscaling',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

# 4. Random forest

Hyperparameters

In [28]:
print(RandomForestClassifier())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators='warn',
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


### Hyperparameter tuning

![RF](img/rf.png)

In [29]:
rf = RandomForestClassifier()
parameters = {
    'n_estimators': [5, 50, 250],
    'max_depth': [2, 4, 8, 16, 32, None]
}

cv = GridSearchCV(rf, parameters, cv=5, n_jobs=-1, verbose=10)
cv.fit(X_train, y_train.values.ravel())

print_results(cv)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  85 out of  90 | elapsed:   18.8s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   21.6s finished


BEST PARAMS: {'max_depth': 16, 'n_estimators': 250}

0.587 (+/-0.042) for {'max_depth': 2, 'n_estimators': 5}
0.561 (+/-0.008) for {'max_depth': 2, 'n_estimators': 50}
0.562 (+/-0.004) for {'max_depth': 2, 'n_estimators': 250}
0.585 (+/-0.043) for {'max_depth': 4, 'n_estimators': 5}
0.595 (+/-0.031) for {'max_depth': 4, 'n_estimators': 50}
0.591 (+/-0.016) for {'max_depth': 4, 'n_estimators': 250}
0.597 (+/-0.028) for {'max_depth': 8, 'n_estimators': 5}
0.627 (+/-0.015) for {'max_depth': 8, 'n_estimators': 50}
0.628 (+/-0.011) for {'max_depth': 8, 'n_estimators': 250}
0.614 (+/-0.015) for {'max_depth': 16, 'n_estimators': 5}
0.641 (+/-0.012) for {'max_depth': 16, 'n_estimators': 50}
0.654 (+/-0.008) for {'max_depth': 16, 'n_estimators': 250}
0.605 (+/-0.029) for {'max_depth': 32, 'n_estimators': 5}
0.64 (+/-0.014) for {'max_depth': 32, 'n_estimators': 50}
0.652 (+/-0.012) for {'max_depth': 32, 'n_estimators': 250}
0.609 (+/-0.024) for {'max_depth': None, 'n_estimators': 5}
0.641 (+/-0.

In [37]:
#rf_best = cv.best_estimator_

with open('RF_best.pickle', 'wb') as f:
    pickle.dump(rf_best, f)

rf_best

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=16, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# 5. Boosting

Hyperparameters

In [31]:
from sklearn.ensemble import GradientBoostingClassifier

print(GradientBoostingClassifier())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


### Hyperparameter tuning

![GB](img/gb.png)

In [32]:
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators': [5, 50, 250, 500],
    'max_depth': [1, 3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 1, 10, 100]
}

cv = GridSearchCV(gb, parameters, cv=5, n_jobs=-1, verbose=10)
cv.fit(X_train, y_train.values.ravel())

print_results(cv)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0648s.) Setting batch_size=6.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Batch computation too slow (2.9641s.) Setting batch_size=3.
[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed:   38.0s
[Parallel(n_jobs=-1)]: Batch computation too slow (38.2243s.) Setting batch_size=1.
[Parallel(n_jobs=-1)]: Done 103 tasks      | elapsed:   55.0s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 172 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 187 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 207 tasks      | elapsed: 

BEST PARAMS: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 500}

0.56 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 5}
0.56 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50}
0.618 (+/-0.008) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 250}
0.622 (+/-0.013) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500}
0.56 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 5}
0.596 (+/-0.01) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
0.634 (+/-0.005) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 250}
0.637 (+/-0.009) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
0.56 (+/-0.0) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 5}
0.616 (+/-0.027) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}
0.64 (+/-0.015) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 250}
0.643 (+/-0.023) for {'learning_rate': 0.01, 'm

In [38]:
#gb_best = cv.best_estimator_

with open('GB_best.pickle', 'wb') as f:
    pickle.dump(gb_best, f)

gb_best

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

# Summary: Compare model results and final model selection

In [43]:
# Read in models
models = {}

for mdl in ['LR', 'SVM', 'MLP', 'RF', 'GB']:
    models[mdl] = pickle.load( open(f'{mdl}_best.pickle', 'rb') ) 

In [44]:
models

{'LR': LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='warn', n_jobs=None, penalty='l2',
                    random_state=None, solver='warn', tol=0.0001, verbose=0,
                    warm_start=False),
 'SVM': SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
     kernel='linear', max_iter=-1, probability=False, random_state=None,
     shrinking=True, tol=0.001, verbose=False),
 'MLP': MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
               beta_2=0.999, early_stopping=False, epsilon=1e-08,
               hidden_layer_sizes=(10,), learning_rate='invscaling',
               learning_rate_init=0.001, max_iter=200, momentum=0.9,
               n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
               random_state=None, shuffle=

### Evaluate models on the validation set

![Evaluation Metrics](img/eval_metrics.png)

In [48]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from time import time

def evaluate_model(name, model, features, labels):
    start = time()
    pred = model.predict(features)
    end = time()
    accuracy = round(accuracy_score(labels, pred), 3)
    precision = round(precision_score(labels, pred), 3)
    recall = round(recall_score(labels, pred), 3)
    print('{} -- Accuracy: {} / Precision: {} / Recall: {} / Latency: {}ms'.format(name,
                                                                                   accuracy,
                                                                                   precision,
                                                                                   recall,
                                                                                   round((end - start)*1000, 1)))

In [49]:
for name, mdl in models.items():
    evaluate_model(name, mdl, X_val, y_val)

LR -- Accuracy: 0.624 / Precision: 0.651 / Recall: 0.377 / Latency: 8.5ms
SVM -- Accuracy: 0.592 / Precision: 0.599 / Recall: 0.313 / Latency: 283.2ms
MLP -- Accuracy: 0.562 / Precision: 0.597 / Recall: 0.114 / Latency: 4.0ms
RF -- Accuracy: 0.66 / Precision: 0.651 / Recall: 0.546 / Latency: 104.7ms
GB -- Accuracy: 0.649 / Precision: 0.623 / Recall: 0.583 / Latency: 26.9ms


### Evaluate best model on test set

In [51]:
evaluate_model('Random Forest', models['RF'], X_test, y_test)
evaluate_model('Gradient Boosting', models['GB'], X_test, y_test)

Random Forest -- Accuracy: 0.659 / Precision: 0.611 / Recall: 0.553 / Latency: 113.2ms
Gradient Boosting -- Accuracy: 0.667 / Precision: 0.612 / Recall: 0.599 / Latency: 24.9ms
